In [ ]:
from flask import Flask, jsonify, request
from flask_cors import CORS
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from io import BytesIO
import requests

app = Flask(__name__)
CORS(app, resources={r"/predict": {"origins": "*"}})

# Load model with compile=False
model = load_model('brain_tumor_model.h5', compile=False)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

@app.route('/predict', methods=['POST', 'OPTIONS'])
def predict():
    if request.method == 'OPTIONS':
        return _build_cors_preflight_response()
        
    data = request.json
    img_url = data['image_url']
    
    try:
        # Download and process image
        response = requests.get(img_url)
        img = image.load_img(BytesIO(response.content), target_size=(150, 150))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array/255.0, axis=0)
        
        # Predict
        predictions = model.predict(img_array)
        class_names = ['glioma', 'meningioma', 'no_tumor', 'pituitary']
        
        return jsonify({
            "class": class_names[np.argmax(predictions[0])],
            "confidence": float(np.max(predictions[0]))
        })
        
    except Exception as e:
        return jsonify({"error": str(e)}), 500

def _build_cors_preflight_response():
    response = jsonify({"message": "Preflight Request"})
    response.headers.add("Access-Control-Allow-Origin", "*")
    response.headers.add("Access-Control-Allow-Headers", "*")
    response.headers.add("Access-Control-Allow-Methods", "*")
    return response

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.209.95:5001
Press CTRL+C to quit
192.168.209.95 - - [22/Mar/2025 10:52:08] "OPTIONS /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step


192.168.209.95 - - [22/Mar/2025 10:52:10] "POST /predict HTTP/1.1" 200 -
192.168.209.95 - - [22/Mar/2025 10:52:34] "OPTIONS /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


192.168.209.95 - - [22/Mar/2025 10:52:36] "POST /predict HTTP/1.1" 200 -
